In [ ]:
from transformers import BertModel, BertConfig, BertForSequenceClassification
#Reading the IMDb trained model
config = BertConfig.from_json_file('/content/drive/MyDrive/Bert/Studentdata/withIMDB/model/config.json')
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Bert/Studentdata/withIMDB/model/pytorch_model.bin', config=config)


In [ ]:
from pathlib import Path
#Reading the Student dataset
split_dir = Path('data_vid')
train_texts = []
train_labels = []
val_texts = []
val_labels = []
test_texts = []
test_labels = []

for label_dir in ["train_pos", "train_neg"]:
    for text_file in (split_dir/label_dir).iterdir():
        tmp = text_file.read_text().split(' ')[1:]
        text = ' '.join([str(elem) for elem in tmp])
        train_texts.append(text)
        train_labels.append(0 if label_dir is "train_neg" else 1)
        
for label_dir in ["dev_pos", "dev_neg"]:
    for text_file in (split_dir/label_dir).iterdir():
        tmp = text_file.read_text().split(' ')[1:]
        text = ' '.join([str(elem) for elem in tmp])
        val_texts.append(text)
        val_labels.append(0 if label_dir is "dev_neg" else 1)
        
for label_dir in ["eval_pos", "eval_neg"]:
    for text_file in (split_dir/label_dir).iterdir():
        tmp = text_file.read_text().split(' ')[1:]
        text = ' '.join([str(elem) for elem in tmp])
        test_texts.append(text)
        test_labels.append(0 if label_dir is "eval_neg" else 1)


In [ ]:
len(train_texts), len(train_labels), len(val_labels), len(val_texts), len(test_labels), len(test_texts)

(298, 298, 209, 209, 211, 211)

In [ ]:
train_texts

In [ ]:
import transformers
from transformers import BertTokenizer
#Tokenizing the datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)


In [ ]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#Training the studentdata on IMDb trained model
training_args = TrainingArguments(
    output_dir='./drive/MyDrive/Bert/EE492/Studentdata/withIMDB/train-val/results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./drive/MyDrive/Bert/EE492/Studentdata/withIMDB/train-val/logs',            # directory for storing logs
    logging_steps=1500,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    compute_metrics=compute_metrics,
    eval_dataset=val_dataset             # evaluation dataset
)



In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=225, training_loss=0.1664262898763021, metrics={'train_runtime': 37.18, 'train_samples_per_second': 6.052, 'total_flos': 30792181088520.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2374787072, 'init_mem_gpu_alloc_delta': 433776128, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 11436032, 'train_mem_gpu_alloc_delta': 1308135936, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 166290944})

In [ ]:
import numpy as np
#Converting prediction values to 0's and 1's basically softmaxing. Then isolating the negatively labelled texts
out = trainer.predict(test_dataset)
neg_arr = np.where(out[0].T[0] >0,0,1)
neg_texts= []
for i in range(len(test_texts)):
  if neg_arr[i] ==  0:
    neg_texts.append(test_texts[i])

neg_texts

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9234449760765551,
 'eval_f1': 0.9327731092436975,
 'eval_loss': 0.46084073185920715,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 25432064,
 'eval_precision': 0.9823008849557522,
 'eval_recall': 0.888,
 'eval_runtime': 1.8784,
 'eval_samples_per_second': 111.266}

In [ ]:
training_args2 = TrainingArguments(
    output_dir='./drive/MyDrive/Bert/LastDay/Studentdata/withIMDB/train-eval/results',          # output directory
    per_device_eval_batch_size=4,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./drive/MyDrive/Bert/Studentdata/withIMDB/train-eval/logs',            # directory for storing logs
    logging_steps=1500,
)

trainer2 = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args2,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    eval_dataset=test_dataset             # evaluation dataset
)

In [ ]:
trainer2.evaluate()

{'eval_accuracy': 0.9052132701421801,
 'eval_f1': 0.9152542372881356,
 'eval_loss': 0.6289113759994507,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 14595584,
 'eval_precision': 0.9818181818181818,
 'eval_recall': 0.8571428571428571,
 'eval_runtime': 1.7415,
 'eval_samples_per_second': 121.158,
 'init_mem_cpu_alloc_delta': 0,
 'init_mem_cpu_peaked_delta': 0,
 'init_mem_gpu_alloc_delta': 0,
 'init_mem_gpu_peaked_delta': 0}

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Bert/EE492/Studentdata/withIMDB/trained/model")